In [85]:
import numpy as np
import pandas as pd
import random
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
random.seed(696)
import matplotlib.pyplot as plt

## Loading dataset and filling NANs

In [63]:
allFiles = glob.glob("Full_data/p*.psv")
patients = pd.DataFrame()
np_array_list = []
patient_id = 0
for file_ in allFiles:
    #print(file_)
    dftmp = pd.read_csv(file_,sep = '|')
    dftmp.fillna(method = 'backfill', inplace=True)
    dftmp.fillna(method = 'ffill', inplace=True)
    dftmp['pid'] = patient_id
    patient_id += 1
    patients = pd.concat([patients, dftmp])

In [65]:
patients.head()
patients = patients.fillna(patients.mean())

In [66]:
patients.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.527362,-2.828916,23.130976,...,304.275007,170.0,73,1,1.0,0.0,-214.64,1,0,0
1,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.527362,-2.828916,23.130976,...,304.275007,170.0,73,1,1.0,0.0,-214.64,2,0,0
2,91.0,96.0,36.5,108.0,84.5,72.0,23.5,33.527362,-2.828916,23.130976,...,304.275007,170.0,73,1,1.0,0.0,-214.64,3,0,0
3,93.0,98.0,36.5,123.0,87.0,61.0,21.0,33.527362,-2.828916,23.130976,...,304.275007,170.0,73,1,1.0,0.0,-214.64,4,0,0
4,93.0,95.0,36.5,110.0,81.0,70.0,20.0,33.527362,-2.828916,23.130976,...,304.275007,170.0,73,1,1.0,0.0,-214.64,5,0,0


In [67]:
nan_stat = patients.isna().sum()/patients.shape[0]
print(nan_stat)

HR                  0.0
O2Sat               0.0
Temp                0.0
SBP                 0.0
MAP                 0.0
DBP                 0.0
Resp                0.0
EtCO2               0.0
BaseExcess          0.0
HCO3                0.0
FiO2                0.0
pH                  0.0
PaCO2               0.0
SaO2                0.0
AST                 0.0
BUN                 0.0
Alkalinephos        0.0
Calcium             0.0
Chloride            0.0
Creatinine          0.0
Bilirubin_direct    0.0
Glucose             0.0
Lactate             0.0
Magnesium           0.0
Phosphate           0.0
Potassium           0.0
Bilirubin_total     0.0
TroponinI           0.0
Hct                 0.0
Hgb                 0.0
PTT                 0.0
WBC                 0.0
Fibrinogen          0.0
Platelets           0.0
Age                 0.0
Gender              0.0
Unit1               0.0
Unit2               0.0
HospAdmTime         0.0
ICULOS              0.0
SepsisLabel         0.0
pid             

In [68]:
y = patients['SepsisLabel'].copy()
pids = patients['pid'].copy()
patients_features = patients.drop('SepsisLabel', axis = 1)
patients_features.drop('pid', axis = 1, inplace = True)
pids.head()

0    0
1    0
2    0
3    0
4    0
Name: pid, dtype: int64

In [69]:
patients_features = (patients_features - patients_features.mean())/patients_features.std()

In [70]:
patients = patients_features
patients['SepsisLabel'] = y.values
patients['pid'] = pids.values
patients.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,0.510602,-1.477257,-0.409269,-0.683074,-0.648402,-0.753586,0.695135,1.172728e-11,-6.898277e-12,-1.164898e-10,...,2.533330e-11,-0.289814,0.764999,0.920766,1.21033,-1.21033,-0.633629,-0.905875,0,0
1,0.510602,-1.477257,-0.409269,-0.683074,-0.648402,-0.753586,0.695135,1.172728e-11,-6.898277e-12,-1.164898e-10,...,2.533330e-11,-0.289814,0.764999,0.920766,1.21033,-1.21033,-0.633629,-0.869032,0,0
2,0.398604,-0.344184,-0.409269,-0.765013,-0.142929,0.369328,1.000835,1.172728e-11,-6.898277e-12,-1.164898e-10,...,2.533330e-11,-0.289814,0.764999,0.920766,1.21033,-1.21033,-0.633629,-0.832190,0,0
3,0.510602,0.303287,-0.409269,-0.150468,0.005740,-0.402675,0.491335,1.172728e-11,-6.898277e-12,-1.164898e-10,...,2.533330e-11,-0.289814,0.764999,0.920766,1.21033,-1.21033,-0.633629,-0.795348,0,0
4,0.510602,-0.667919,-0.409269,-0.683074,-0.351065,0.228964,0.287535,1.172728e-11,-6.898277e-12,-1.164898e-10,...,2.533330e-11,-0.289814,0.764999,0.920766,1.21033,-1.21033,-0.633629,-0.758506,0,0


## Split data

In [71]:
def get_split_indices(pos_len, neg_len, ratios):
    train, val, test = ratios[0], ratios[1], ratios[2]
    pos_tr, neg_tr = int(round(pos_len * train)), int(round(neg_len * train))
    pos_val, neg_val = int(round(pos_len * val)), int(round(neg_len * val))
    pos_test, neg_test = int(round(pos_len * test)), int(round(neg_len * test))
    return ((pos_tr, pos_val, pos_test), (neg_tr, neg_val, neg_test))
    
    
def split_dataset(patients, ratios):
    positive = patients[patients['SepsisLabel'] == 1]['pid'].unique().tolist()
    negative = [i for i in range(1, 5000+1) if i not in positive]
    random.shuffle(positive)
    random.shuffle(negative)
    pos_idx, neg_idx = get_split_indices(len(positive), len(negative), ratios)
    
    train = positive[0:pos_idx[0]] + negative[0:neg_idx[0]]
    val = positive[pos_idx[0]:pos_idx[0] + pos_idx[1]] + negative[neg_idx[0]:neg_idx[0] + neg_idx[1]]
    test = positive[pos_idx[0] + pos_idx[1]:] + negative[neg_idx[0] + neg_idx[1]:]
    
    train_dict, val_dict, test_dict = {}, {}, {}
    for pid in train:
        train_dict[pid] = patients[patients['pid'] == pid]
    for pid in val:
        val_dict[pid] = patients[patients['pid'] == pid]
    for pid in test:
        test_dict[pid] = patients[patients['pid'] == pid]
    
    return train_dict, val_dict, test_dict

## Choose patient observation windows

In [72]:
def process_patient(patient, max_len, window_marker=70):
    patient = patient.reset_index()
    obs_len = patient.shape[0]
    
    if(patient[patient['SepsisLabel'] == 1].shape[0]):
        sepsis_idx = list(patient[patient['SepsisLabel']==1].index)[0]
        if obs_len > max_len: return process_longer_obs(patient, sepsis_idx, max_len, marker=window_marker)
        if obs_len < max_len: return process_shorter_obs(patient, sepsis_idx, max_len)
    else:
        if obs_len > max_len:
            return patient.iloc[0:max_len, 2:]
        if obs_len < max_len:
            p = patient.iloc[:, :]
            for i in range(max_len - obs_len):
                p = p.append(patient.iloc[-1, :])
            return p.iloc[:, 2:]
    return patient.iloc[:, 2:]

def process_shorter_obs(patient, sepsis_idx, max_len):
    p = pd.DataFrame()
    p = p.append(patient)
    for i in range(max_len - patient.shape[0]):
        p = p.append(p.iloc[-1, :])
    return p.reset_index().iloc[:, 3:]

def process_longer_obs(patient, sepsis_idx, max_len, marker=70):
    p = pd.DataFrame()
    avail_before = sepsis_idx - 1
    avail_after = patient.shape[0] - sepsis_idx
    need_before = int(max_len * marker/100)
    need_after = int(max_len * (100 - marker)/100) - 1
   
    if avail_before >= need_before and avail_after >= need_after:
        p = p.append(patient.iloc[avail_before - need_before:avail_before+1, :])
        p = p.append(patient.iloc[sepsis_idx, :])
        p = p.append(patient.iloc[sepsis_idx+1 : sepsis_idx + need_after, :])
    
    elif avail_before >= need_before and avail_after <= need_after:
        p = p.append(patient.iloc[avail_before - need_before:avail_before+1, :])
        p = p.append(patient.iloc[sepsis_idx, :])
        p = p.append(patient.iloc[sepsis_idx + 1:, :])
        for i in range(max_len - p.shape[0]):
            p = p.append(p.iloc[-1, :])
    
    elif avail_before <= need_before and avail_after >= need_after:
        p = p.append(patient.iloc[0:avail_before, :])
        p = p.append(patient.iloc[sepsis_idx, :])
        p = p.append(patient.iloc[sepsis_idx+1 : sepsis_idx + need_after, :])
        for i in range(max_len - p.shape[0]):
            p = p.concat([p.iloc[0, :], p], ignore_index = True)
    
    return p.reset_index().iloc[:, 3:]

## Create DataLoaders

In [73]:
class PatientDataset(Dataset):
    def __init__(self, patient_dict, max_obs_len, window_marker):
        self.patient_dict = patient_dict
        self.num_patients = len(patient_dict)
        self.pids = list(patient_dict.keys())
        self.max_obs_len = max_obs_len
        self.window_marker = window_marker
        
    def __len__(self):
        return self.num_patients
    
    def __getitem__(self, idx):
        patient = self.patient_dict[self.pids[idx]]
        patient = process_patient(patient, self.max_obs_len, self.window_marker)
        patient_features = torch.FloatTensor(patient.iloc[:, 1:-1].values)
        patient_labels = torch.FloatTensor(patient['SepsisLabel'])
        self.num_patients -= 1
        return patient_features, patient_labels

In [74]:
def data_loader(patient_dict, max_obs_len, batch_size, shuffle=True, window_marker=70):
    return DataLoader(PatientDataset(patient_dict, max_obs_len, window_marker), batch_size, shuffle)

## Model training and evaluation setup

In [75]:
def confusion_matrix(prediction, truth):
    confusion_vector = prediction/truth
    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()
    return true_positives, false_positives, true_negatives, false_negatives

def check_accuracy(model, loader, group):
    print('Checking ' + group + ' accuracy!')
    num_correct = 0
    num_samples = 0
    tp, fp, tn, fn, precision, recall, f1 = 0, 0, 0, 0, 0 ,0, 0
    model.eval()
    for t, (x, y) in enumerate(loader):
        scores = model(x)
        rounded_preds = torch.round(torch.sigmoid(scores))
        num_correct += (rounded_preds == y).sum()
        num_samples += y.size(0) * y.size(1)
        tp_t, fp_t, tn_t, fn_t = confusion_matrix(rounded_preds, y)
        tp += tp_t
        fp += fp_t
        tn += tn_t
        fn += fn_t

    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    print('TP = ', tp, ', FP = ', fp, ', TN = ', tn, ', FN = ', fn)
    if tp != 0:
        precision = tp/(tp + fp)
        recall = tp/(tp + fn)
        f1 = 2 * ((precision * recall)/(precision + recall))
    print('Precision = ', precision, ', Recall = ', recall, ', F1 Score = ', f1)
    print()
    return (100*acc, precision, recall)

In [77]:
def train(model, optimizer, loss_fn, train_dict, val_dict, max_obs_len, batch_size, epochs=1, print_every=50, window_marker=70):
    train_history, val_history = {}, {}
    train_history["accuracy"] = []
    train_history["precision"] = []
    train_history["recall"] = []
    
    val_history["accuracy"] = []
    val_history["precision"] = []
    val_history["recall"] = []
    
    for e in range(epochs):
        print('Epoch: ', e+1)
        for t, (x, y) in enumerate(data_loader(train_dict, max_obs_len, batch_size, window_marker)):
            model.train()
            scores = model(x)
            loss = loss_fn(scores, y)
            
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.item()))
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        t_acc, t_precision, t_recall = check_accuracy(model, data_loader(train_dict, max_obs_len, batch_size, window_marker), 'train')
        v_acc, v_precision, v_recall = check_accuracy(model, data_loader(val_dict, max_obs_len, batch_size, window_marker), 'val')
        train_history["accuracy"].append(t_acc)
        train_history["precision"].append(t_precision)
        train_history["recall"].append(t_recall)
        
        val_history["accuracy"].append(v_acc)
        val_history["precision"].append(v_precision)
        val_history["recall"].append(v_recall)
        print()
    return (train_history, val_history)

In [78]:
def get_pos_weight(patient_dict):
    subset = patients[patients['pid'].isin(list(patient_dict.keys()))]
    total_samples = len(subset)
    pos_samples = subset[subset['SepsisLabel'] == 1]['pid'].count()
    return total_samples/pos_samples

## Simple LSTM model

In [79]:
class SimpleLSTM(nn.Module):
    def __init__(self, feature_dim, hidden_dim, out_dim):
        super().__init__()
        self.rnn = nn.RNN(feature_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, out_dim)
        
    def forward(self, x):
        output, hidden = self.rnn(x)
        fc_out = self.fc(hidden.squeeze(0))        
        return fc_out

In [80]:
def test_run(config):
    train_dict, val_dict, test_dict = split_dataset(patients, config['ratios'])
    feature_dim, hidden_dim, output_dim = config['feature_dim'], config['hidden_dim'], config['output_dim']
    model = SimpleLSTM(feature_dim, hidden_dim, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=config['lr_rate'])
    criterion = nn.BCEWithLogitsLoss()
    if config['pos_weight'] is not None:
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([config['pos_weight']]))
    
    train_hist, val_hist = train(model, optimizer, criterion, 
                                     train_dict, val_dict, config['max_obs_len'], config['batch_size'], 
                                     epochs=config['epochs'], window_marker=config['window_marker'])
    return model, train_hist, val_hist 

In [84]:
model_config = {
    'ratios': (.7, .2, .1),
    'feature_dim': 39,
    'hidden_dim': 128,
    'output_dim': 30,
    'max_obs_len': 30,
    'batch_size': 16,
    'lr_rate': 1e-3,
    'pos_weight': None,
    'epochs': 5,
    'window_marker': 70
}

model, train_hist, val_hist = test_run(model_config)

Epoch:  1
t = 50, loss = 0.3185
t = 100, loss = 0.0320
t = 150, loss = 0.0262
t = 200, loss = 0.0466
Checking train accuracy!
Got 102025 / 105000 correct (97.17)
TP =  369 , FP =  233 , TN =  101656 , FN =  2742
Precision =  0.6129568106312292 , Recall =  0.11861137897782063 , F1 Score =  0.1987611096148667

Checking val accuracy!
Got 29074 / 30000 correct (96.91)
TP =  20 , FP =  62 , TN =  29054 , FN =  864
Precision =  0.24390243902439024 , Recall =  0.02262443438914027 , F1 Score =  0.041407867494824016


Epoch:  2
t = 50, loss = 0.0090
t = 100, loss = 0.4000
t = 150, loss = 0.0689
t = 200, loss = 0.0257
Checking train accuracy!
Got 102976 / 105000 correct (98.07)
TP =  1139 , FP =  52 , TN =  101837 , FN =  1972
Precision =  0.9563392107472712 , Recall =  0.366120218579235 , F1 Score =  0.5295211529521153

Checking val accuracy!
Got 29312 / 30000 correct (97.71)
TP =  207 , FP =  11 , TN =  29105 , FN =  677
Precision =  0.9495412844036697 , Recall =  0.2341628959276018 , F1 Score

In [89]:
model_config = {
    'ratios': (.7, .2, .1),
    'feature_dim': 39,
    'hidden_dim': 128,
    'output_dim': 30,
    'max_obs_len': 30,
    'batch_size': 16,
    'lr_rate': 1e-3,
    'pos_weight': None,
    'epochs': 10,
    'window_marker': 70
}

model, train_hist, val_hist = test_run(model_config)

Epoch:  1
t = 50, loss = 0.2090
t = 100, loss = 0.3659
t = 150, loss = 0.1636
t = 200, loss = 0.1135
Checking train accuracy!
Got 102427 / 105000 correct (97.55)
TP =  635 , FP =  97 , TN =  101792 , FN =  2476
Precision =  0.8674863387978142 , Recall =  0.20411443265830922 , F1 Score =  0.3304709862086911

Checking val accuracy!
Got 29202 / 30000 correct (97.34)
TP =  99 , FP =  13 , TN =  29103 , FN =  785
Precision =  0.8839285714285714 , Recall =  0.11199095022624435 , F1 Score =  0.19879518072289154


Epoch:  2
t = 50, loss = 0.1220
t = 100, loss = 0.0269
t = 150, loss = 0.0370
t = 200, loss = 0.0061
Checking train accuracy!
Got 103772 / 105000 correct (98.83)
TP =  1958 , FP =  75 , TN =  101814 , FN =  1153
Precision =  0.9631087063453025 , Recall =  0.6293796207007393 , F1 Score =  0.7612752721617418

Checking val accuracy!
Got 29563 / 30000 correct (98.54)
TP =  472 , FP =  25 , TN =  29091 , FN =  412
Precision =  0.9496981891348089 , Recall =  0.5339366515837104 , F1 Score =